# Imports & Setup

In [1]:
import datetime as dt
import time
import random
import logging
import numpy as np
from collections import defaultdict
from collections import deque
import predict

from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick

from IPython.display import clear_output

logging.getLogger('client').setLevel('ERROR')

# Function definitions

In [19]:
def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='ioc',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='ioc',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)
        
def is_liquid(bid_prices, ask_prices, trading_volume_bids, trading_volume_asks, liquidity_threshold=0.1, unbalance_threshold=0.2):
    # Calculate the weighted average bid and ask prices
    weighted_avg_bid_price = sum(bid * volume for bid, volume in zip(bid_prices, trading_volume_bids)) / sum(trading_volume_bids)
    weighted_avg_ask_price = sum(ask * volume for ask, volume in zip(ask_prices, trading_volume_asks)) / sum(trading_volume_asks)

    # Calculate the bid-ask spread using the weighted average prices
    bid_ask_spread = weighted_avg_ask_price - weighted_avg_bid_price

    # Calculate the total buying and selling volumes
    total_bid_volume = sum(trading_volume_bids)
    total_ask_volume = sum(trading_volume_asks)

    # Improved Spread-to-Volume Ratio
    spread_to_volume = bid_ask_spread / (total_bid_volume + total_ask_volume)

    # Calculate the order imbalance
    order_imbalance = abs(total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)

    # Check for market liquidity based on the liquidity threshold
    is_liquid = spread_to_volume <= liquidity_threshold

    # Check for market unbalance based on the unbalance threshold
    is_unbalanced = order_imbalance >= unbalance_threshold

    print(f'is_liquid {is_liquid}, is_unbalanced {is_unbalanced}')
    return is_liquid, is_unbalanced


    
    

In [20]:
def calculate_direction(rsi_indicator,bid_ask_indicator,sentiment_score,last_tweet_dict,instrument_id):
    
    print('last_tweet_dict %s',last_tweet_dict)
    
    iteration_number = 0
    
    if instrument_id in last_tweet_dict.keys():
        sentiment_score, iteration_number = last_tweet_dict[instrument_id]
    
    if (iteration_number>0):
        last_tweet_dict[instrument_id] = [sentiment_score,iteration_number-1]
    else:
        sentiment_score = 0.5
    
    # Sample weights for each factor (you should adjust these based on relevance)
    weight_sentiment = 0.5
    weight_volume = 0.25
    weight_rsi = 0.25
    
    if sentiment_score > 0.01:
        ss2 = 1
    elif sentiment_score < -0.01:
        ss2=0
    else:
        ss2 = 0.5
    
    # Calculate the weighted sum to determine the direction
    weighted_sum = (
        rsi_indicator * weight_rsi +
        bid_ask_indicator * weight_volume +
        ss2 * weight_sentiment

    )
    
    print('sentiment score %s', sentiment_score)
    print('volume_difference score %s', bid_ask_indicator)
    print('market_indicators (rsi) score %s',rsi_indicator)

    # Define a threshold for the gradient (you can adjust this based on your criteria)
    gradient_threshold = 0.5  # Example threshold: 0.6 (neutral position)

    print('weighted_sum gradient %s',weighted_sum)
    # Estimate market maker's position direction based on the gradient
    if weighted_sum > gradient_threshold:
        direction = "Long"  # Positive gradient
    elif weighted_sum < gradient_threshold:
        direction = "Short"  # Negative gradient
    else:
        direction = "Neutral"  # Neutral gradient

    print(f"Market Maker Position Direction: {direction}")
    
    return direction, weighted_sum


In [21]:
def sliding_window(rolling_window, current_price):
    
    # Add the new element to the right side of the deque
    rolling_window.append(current_price)
    
    return rolling_window

In [22]:
def calculate_ema(data, period):
    """
    Calculate the Exponential Moving Average (EMA) for a given dataset and period.
    
    Parameters:
    - data (list): The dataset for which the EMA should be calculated.
    - period (int): The period for the EMA.
    
    Returns:
    - list: The EMA for the dataset.
    """
    
    if len(data) < period:
        # Return simple average for datasets smaller than the period
        return [sum(data) / len(data)] * len(data)

    multiplier = 2 / (period + 1)
    ema = [data[0]]  # Initialize EMA with the first data point

    for i in range(1, len(data)):
        ema_value = (data[i] - ema[-1]) * multiplier + ema[-1]
        ema.append(ema_value)

    return ema


In [23]:
def calculate_rsi(data, period):
    if len(data) < period:
        return 0.5  # Return a neutral value if there isn't enough data for the specified period

    gains = []
    losses = []

    for i in range(1, len(data)):
        price_difference = data[i] - data[i - 1]
        if price_difference >= 0:
            gains.append(price_difference)
            losses.append(0)
        else:
            gains.append(0)
            losses.append(abs(price_difference))

    average_gain = sum(gains[:period]) / period
    average_loss = sum(losses[:period]) / period

    if average_loss == 0:
        return 1.0  # Return 1.0 if there are no losses, indicating a very strong upward trend

    relative_strength = average_gain / average_loss
    rsi = 100 - (100 / (1 + relative_strength))
    
    return rsi / 100  # Normalize RSI to a value between 0 and 1


In [24]:
def calculate_bid_ask_indicator(bid_volume, ask_volume):
    # Ensure bid_volume and ask_volume are non-negative
    bid_volume = max(0, bid_volume)
    ask_volume = max(0, ask_volume)

    # Calculate the bid-ask indicator
    total_volume = bid_volume + ask_volume
    if total_volume == 0:
        return 0.5  # Default to 0.5 if total volume is zero (neutral)

    bid_ask_indicator = bid_volume / (bid_volume + ask_volume)
    
    # Ensure the result is between 0 and 1
    return max(0, min(1, bid_ask_indicator))


In [32]:
def get_tweet_impact_score(feed,instrument_id,last_tweet_dict):
    if len(feed)>0:
        tweet = feed[0].post
        print("tweet --" + tweet)
        prediction = predict.predict(tweet)
        
        if prediction is not None:
            if prediction[1] is not None:
                print("impact -- " +str(prediction[1]))

                last_tweet_dict[prediction[0]] = [(prediction[1]+1)/2 ,500]
                return prediction[1]   
    else:
        return 0.5

# Main algorithm

In [ ]:
# Initialize the exchange
exchange = Exchange()
exchange.connect()

INSTRUMENTS = exchange.get_instruments()

# Constants
QUOTED_VOLUME = 3
FIXED_MINIMUM_CREDIT = 0.1
POSITION_LIMIT = 100
MICHAEL_JORDAN_WINDOW = 223
MICHAEL_JORDAN_TWEET_ITERATIONS = 500

# Data structures to hold stock data and tweet impacts
queue_stock = defaultdict(lambda: deque(maxlen=MICHAEL_JORDAN_WINDOW))
last_tweet_dict = defaultdict()

while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    # Display our own current positions in all stocks, and our PnL so far
    print_positions_and_pnl(exchange)
    print(f'')
    print(f'          (ourbid) mktbid :: mktask (ourask)')

    for instrument in INSTRUMENTS.values():
        # Get the impact of any new tweets
        tweet_impact = get_tweet_impact_score(exchange.poll_new_social_media_feeds(), instrument.instrument_id, last_tweet_dict)

        # Get the current order book for the instrument
        instrument_order_book = exchange.get_last_price_book(instrument.instrument_id)

        # Check for incomplete order books and skip if necessary
        if not (instrument_order_book and instrument_order_book.bids and instrument_order_book.asks):
            print(f'{instrument.instrument_id:>6s} --     INCOMPLETE ORDER BOOK')
            continue

        # Extract bid and ask prices and volumes
        bids = [bid.price for bid in instrument_order_book.bids]
        asks = [ask.price for ask in instrument_order_book.asks]
        volume_bids = [bid.volume for bid in instrument_order_book.bids]
        volume_asks = [ask.volume for ask in instrument_order_book.asks]

        # Calculate the mid price
        mid_price = (bids[0] + asks[0]) / 2
        print('mid price %s', mid_price)

        # Update the sliding window with the current price
        sliding_window_var = sliding_window(queue_stock[instrument.instrument_id], mid_price)

        # Calculate the RSI and bid-ask indicator
        rsi = calculate_rsi(sliding_window_var, MICHAEL_JORDAN_WINDOW)
        bid_ask_indicator = calculate_bid_ask_indicator(sum(volume_bids), sum(volume_asks))
        # Determine the trading direction based on RSI, bid-ask indicator, and tweet impact
        direction, gradient = calculate_direction(rsi, bid_ask_indicator, tweet_impact, last_tweet_dict, instrument.instrument_id)
        print('direction %s', direction)

        # Check if the market is liquid and not unbalanced
        liquid, unbalanced = is_liquid(bids, asks, volume_bids, volume_asks)

        if liquid and not unbalanced:
            # Adjust bid and ask prices based on EMA, current position, and tweet impact
            position = exchange.get_positions()[instrument.instrument_id]
            ema = calculate_ema(sliding_window_var, MICHAEL_JORDAN_WINDOW)
            if ema[-1] > mid_price:  # EMA is trending up
                if position > 0:  # We have a long position
                    bid_price = mid_price + FIXED_MINIMUM_CREDIT + tweet_impact
                    ask_price = mid_price + 2 * FIXED_MINIMUM_CREDIT + tweet_impact
                else:
                    bid_price = mid_price + FIXED_MINIMUM_CREDIT
                    ask_price = mid_price + 2 * FIXED_MINIMUM_CREDIT
            else:  # EMA is trending down
                if position < 0:  # We have a short position
                    bid_price = mid_price - FIXED_MINIMUM_CREDIT - tweet_impact
                    ask_price = mid_price - 2 * FIXED_MINIMUM_CREDIT - tweet_impact
                else:
                    bid_price = mid_price - FIXED_MINIMUM_CREDIT
                    ask_price = mid_price - 2 * FIXED_MINIMUM_CREDIT

            # Insert new quotes
            max_volume_to_buy = POSITION_LIMIT - position
            max_volume_to_sell = POSITION_LIMIT + position
            bid_volume = min(QUOTED_VOLUME, max_volume_to_buy)
            ask_volume = min(QUOTED_VOLUME, max_volume_to_sell)
            insert_quotes(exchange, instrument, round(bid_price, 1), round(ask_price, 1), bid_volume, ask_volume)
        else:
            print('The market is illiquid or unbalanced')

        # Wait for a few seconds to refresh the quotes
        print(f'\nWaiting for 2 seconds.')
        time.sleep(0.05)

    # Clear the displayed information after waiting
    clear_output(wait=True)



-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2023-09-17 04:31:32.151058 UTC.
-----------------------------------------------------------------
Positions:
  NVDA      :  -22
  ING       :  -61
  SAN       : -100
  PFE       :    2
  CSCO      :  -82

PnL: -20696.20

          (ourbid) mktbid :: mktask (ourask)
mid price %s 53.800000000000004
last_tweet_dict %s defaultdict(None, {'CSCO': [0.5121608339250088, 483]})
sentiment score %s 0.5121608339250088
volume_difference score %s 0.5892193308550185
market_indicators (rsi) score %s 0.5
weighted_sum gradient %s 0.7723048327137546
Market Maker Position Direction: Long
direction %s Long
is_liquid True, is_unbalanced False

Waiting for 2 seconds.


2023-09-17 04:31:32,345 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443


tweet --@TechTrends: Cisco's stock hit as executives miss a tech seminar due to scheduling conflicts. Adjusting inbound strategy? #TechNews #SchedulingConflicts


2023-09-17 04:31:32,476 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
2023-09-17 04:31:32,483 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-09-17 04:31:32,673 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
